In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import warnings 
warnings.filterwarnings('ignore')
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
options = webdriver.ChromeOptions()
# options.add_argument(r"C:\Users\Asif-LFD\AppData\Local\Google\Chrome\User Data\Profile 1")
driver = webdriver.Chrome(ChromeDriverManager().install(),options=options)
# from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import datetime



In [ ]:
url = "https://pk.indeed.com/"
driver.get(url)
time.sleep(2)

# input keyword (input variable may be 1. job title 2. where) 
1. clear() kaam nahi kar rha dyk lyna bad me

In [ ]:
var_key_word_job = "Data Engineer"
var_key_word_where = "karachi"
var_btn_keyword = driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/div/div[1]/div/div[1]/div/div[2]/input")
var_btn_keyword.clear()
var_btn_keyword.send_keys(var_key_word_job)
time.sleep(2)

var_btn_keyword = driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/div/div[2]/div/div[1]/div/div[2]/input")
var_btn_keyword.clear()
var_btn_keyword.send_keys(var_key_word_where)
time.sleep(2)

# click find jon button
driver.find_element("xpath","/html/body/div/div[1]/div/span/div[3]/div[1]/div/div/div/div/form/button").click()


In [ ]:
print("There are "+ driver.find_element("xpath","/html/body/main/div/div[1]/div/div/div[5]/div[1]/div[4]/div/div/div[2]/span[1]").text)

# left side me companies ki details jo ati list wise os ka code

1. div (id = 'mosaic-provider-jobcards') k andar "ul" ka tag id
2. ul k andar "li" ka tag {har li me indivdual company ka card}
3. li k andar div class "job_seen_beacon" os k andar sara data






In [ ]:
var_cards_company_data = driver.find_element("xpath","//div[@id='mosaic-provider-jobcards']").find_element("xpath","ul/li/div").find_elements("xpath","//div[@class='slider_container css-8xisqv eu4oa1w0']")
len(var_cards_company_data)

print(len(var_cards_company_data))

# saving all data into csv
var_csv_cols = ['job_title','company_name',	'location',	'salary_range',	'job_type',	'description',	'job_link',	'indeedApply',"hiringMultipleCandidates","posted_date"]


# makinf df having columns of var_csv_cols
df = pd.DataFrame(columns=var_csv_cols)

In [ ]:
for i in range(len(var_cards_company_data)):
# for i in range(6,7):


    print(i)
    time.sleep(5)
    
    # clicking on each card company name
    var_cards_company_data[i].click()
    time.sleep(5)
    # temp
    # soup = BeautifulSoup(var_cards_company_data[10].get_attribute('innerHTML'), 'html.parser')
    # print(soup.prettify())

    try:
        # find salary
        salary_range=var_cards_company_data[i].find_element("xpath","//table[@class='jobCard_mainContent big6_visualChanges']").find_element("xpath","//tr").find_element("xpath","td").find_element("xpath","div[@class='heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly']").find_element("xpath","div[@class='metadata salary-snippet-container']").text
    except:
        salary_range = "undefine"
    salary_range

    # easyapplu and other tags
    soup = BeautifulSoup(var_cards_company_data[i].find_element("xpath","//tr[@class = 'jobCardShelf']").get_attribute('innerHTML'), 'html.parser')
    # print(soup.prettify())
    html = str(soup.prettify())
    regex = r'td class="shelfItem ([^"]*)">'
    matches = re.findall(regex, html)
    next_words = [match.strip() for match in matches]
    # print(next_words)

    # posted date
    posted_date = var_cards_company_data[i].find_element("xpath","//table[@class = 'jobCardShelfContainer big6_visualChanges']").find_element("xpath","//tr[@class = 'underShelfFooter']").find_element("xpath","//span[@class='date']").text.split("\n")[-1]
    posted_date

    #  right portion
    # page ka right side
    var_right_side = driver.find_element("xpath","//div[@class = 'jobsearch-JobComponent icl-u-xs-mt--sm jobsearch-JobComponent--embedded css-axjo09 eu4oa1w0']")

    #ab yaha sy user interction(kisi button py click karna etc etc) kahtam hun gaya just data scrape karna han so we need to use bs4. 
    # bs4
    soup = BeautifulSoup(var_right_side.get_attribute('innerHTML'), 'html.parser')
    # print(soup.prettify())



    ### right up portion (jis me job title,comapny name, location and apply karny ka button han)
    # print(soup.find('div',class_="jobsearch-InfoHeaderContainer").prettify())

    # job title
    job_title = soup.find('h2',class_='icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title is-embedded').text.replace(" - job post","")
    # job_title

    # comapny name
    company_name = soup.find("div",class_="css-1h46us2 eu4oa1w0").text #comapny name
    # company_name

    # location
    location = soup.find("div",class_="css-6z8o9s eu4oa1w0").text #location
    location

    # easyapply
    if (soup.find("span",class_='jobsearch-IndeedApplyButton-newDesign css-1hjxf1u eu4oa1w0').text == 'Apply now'):
        indeedApply = 1
    else:
        indeedApply = 0

    
    

    # nechy waly code me 2 divs aa rahy. may be contry ka hun. lakin abhi pata nahi q future me kaam aa skta
    # soup.find_all("div",class_="css-6z8o9s eu4oa1w0")

    ### right down portion (jis me job description wagera han)
    var_soup_job_des = soup.find("div",class_='jobsearch-embeddedBody css-1omm75o eu4oa1w0')
    var_soup_job_des

    try:
        # job type
        job_type = var_soup_job_des.find_all("div",class_="css-tvvxwd ecydgvn1")
        # .text
        for divs in job_type:
            if divs.text() == "Job Type":
                job_type = divs.find("div",class_="css-1t5f0fr ecydgvn0").text
                break
        # job_type



    except:
        job_type = ""


    # jobDescriptionText
    description = var_soup_job_des.find("div",id="jobDescriptionText").text

    # # save in text file
    # with open("extracted_data_"+str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S"))+".txt","w",encoding="utf-8") as f:
    #     f.write(var_soup_job_des.find("div",id="jobDescriptionText").text)
    #     f.close()
    # description

    # getting link
    job_link = driver.current_url
    job_link

    # making an empty dictionary on above listt
    var_dict = {i:'' for i in var_csv_cols}
    var_dict

    # # var_dict[i]
    # next_words
    # # if next_words have any of var_dict keys than put 1 of each keys values in var_dic
    # for i in next_words:
    #     if i in var_dict.keys():
    #         var_dict[i] = 1
    #     else:
    #         var_dict[i] = 0

    # var_dict
    # var_dict.keys()
    # description
    # var_dict["description"] = description
    # var_dict


    var_dict["job_title"] =job_title
    var_dict["company_name"] =company_name
    var_dict["location"] =location
    var_dict["salary_range"] =salary_range
    var_dict["job_type"] =job_type
    var_dict["description"] = description
    var_dict["job_link"] =job_link
    var_dict["posted_date"] =posted_date


    # print(var_dict)

    # adding var_dict to df
    df = df.append(var_dict,ignore_index=True)
    # df


In [ ]:
var_soup_job_des.find_all("div",class_="css-fhkva6 eu4oa1w0")[0].text

In [ ]:
len(var_soup_job_des.find_all("div",class_="css-fhkva6 eu4oa1w0"))

In [ ]:
#     try:
job_type = ""

job_types = var_soup_job_des.find_all("div",class_="css-fhkva6 eu4oa1w0")
# var_soup_job_des.find_all("div",class_="css-tvvxwd ecydgvn1")

for divs in range(len(job_types)):
        # print(divs.text)

        if var_soup_job_des.find_all("div",class_="css-fhkva6 eu4oa1w0")[divs].text == "Job Type":
                job_type = var_soup_job_des.find_all("div",class_="css-tvvxwd ecydgvn1")[divs].text
                break
job_type

In [ ]:
company_name

In [ ]:
# saving file with current time with seconds
df.to_csv("extracted_data_"+str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S"))+".csv",index=False)
# df.to_csv("extracted_data.csv",index=False)

In [ ]:
# left side div class = "css-tvvxwd ecydgvn1"

In [ ]:
# import re

# def keyword_matching(resume, job_description):
#     resume_keywords = set(re.findall(r'\b\w+\b', resume.lower()))
#     job_keywords = set(re.findall(r'\b\w+\b', job_description.lower()))
#     matched_keywords = resume_keywords.intersection(job_keywords)
#     return matched_keywords

# # Example usage
# resume_text = '''

# EHTISHAM ALI
# 03412373830 | etsali232@gmail.com | linkedin.com/in/ali-ehtisham/ | github.com/ehtish
# Objective
# Skilled web developer with expertise in HTML, CSS, JavaScript, Bootstrap, and React. Passionate about creating visually stunning and user-friendly web experiences. Seeking a challenging role to leverage MERN stack proficiency and drive impactful solutions that exceed client expectations.
# Experience
# CONTOUR SOFTWARE
# MERN stack trainee
# - During the comprehensive MERN (MongoDB, Express, React, Node) stack development training, hands-on experience was gained in building and deploying responsive web applications.
# - Worked under the supervision of industry experts to provide solutions to real-world problems.
# - Developed proficiency in front-end development, creating dynamic user interfaces using React.
# - Furthermore, valuable knowledge and extensive practical experience were diligently acquired in API building and proficiently handling hosting operations.
# Projects
# 1. Exercise Tracker (Final Project at CONTOUR)
# - The Exercise Tracker project utilizes the MERN stack, including MongoDB, Express, React, and Node.js, along with Bootstrap and Material UI. It offers a visually appealing, user-friendly experience with a responsive design, catering to various devices. Secure login is implemented using JWT tokens for seamless activity management.
# 2. Managee
# - Efficiently developing the front-end of the software team management website, the project showcased proficient utilization of HTML, CSS, JavaScript, jQuery, and the feature-rich JavaScript library, Owl Carousel. With a strong emphasis on responsive design, the website ensures a seamless user experience and intuitive features.
# - Live demo: https://ehtish.github.io/managee/
# 3. Mr. Mashwara
# - Developed a chrome extension project utilizing the advice API with skillful implementation of HTML5, CSS3, JavaScript, and JSON.
# - Live demo: https://github.com/Ehtish/Mr.Mashwara
# 4. Fylo
# - Skillfully created Fylo, a file securing website, utilizing HTML5 and CSS3 with a special feature of dynamically responsive font size functionality, showcasing technical expertise and innovation.
# - Live demo: https://ehtish.github.io/fylo-dark/
# 5. Portfolio
# - Recently collaborated on a dynamic portfolio project that demonstrated proficiency in HTML5, CSS3, Bootstrap 4, and JavaScript. Impressively showcased front-end skills through custom styling, responsive design, and engaging interactive features.
# - Live demo: https://ehtish.github.io/asif-portfolio/
# - Note: Explore more projects and contributions on my GitHub profile: https://github.com/Ehtish
# Academics
# BSCS
# SINDH MADRASATUL ISLAM UNVERSITY
# Jan 2016 -2020
# Skills
# - HTML5, CSS3, Javascript, Bootstrap, React, Node, Express, and MongoDB.
# Soft Skills
# - Communication, Problem Solving, Adaptability, Teamwork and Time Management.
# Platforms - Git, GitHub, Netlify and MySQL.
# Beyond Academics - Reading Books and Gaming.
# '''

# job_description_text = '''
# A Frontend Developer is responsible for creating and implementing the visual and interactive elements of a website or application. They work closely with designers, back-end developers, and stakeholders to ensure that the user experience is seamless and intuitive. Frontend Developers use a combination of programming languages and tools to bring design concepts to life, and must be able to balance technical skills with creativity and attention to detail.

# Key Responsibilities:
# - Collaborate with designers, back-end developers, and stakeholders to develop and implement user-friendly interfaces and interactive features
# - Write clean, efficient, and well-documented code using HTML, CSS, and JavaScript
# - Hands-on experience with JS libraries and frameworks like ReactJS, VueJS, Angular
# - Should be able to connect APIs with frontend shared by backend team
# - Optimize web pages for maximum speed and scalability
# - Ensure cross-browser and cross-device compatibility
# - Identify and troubleshoot issues related to performance, user experience, and accessibility
# - Stay up-to-date with emerging trends and technologies in web development
# - Conduct usability testing and gather feedback from users to improve website or application design
# - Communicate technical information to non-technical stakeholders

# Qualifications:
# - Bachelor's degree in Computer Science, Software Engineering, or related field
# - Proven experience as a Frontend Developer or similar role
# - Proficiency in HTML, CSS, JavaScript, and related frameworks such as React, Angular, or Vue.js
# - Experience with responsive design, mobile-first development, and cross-browser compatibility
# - Knowledge of user experience (UX) design principles and methodologies
# - Familiarity with version control tools such as Git
# - Strong problem-solving and analytical skills
# - Excellent communication and teamwork abilities

# Preferred Qualifications:
# - Experience with web performance optimization techniques
# - Familiarity with server-side scripting languages such as PHP or Node.js
# - Knowledge of search engine optimization (SEO) best practices
# - Experience with web accessibility standards and guidelines (WCAG)
# - Strong portfolio showcasing previous web development projects

# Job Type: Full-tim
# '''

# matched_keywords = keyword_matching(resume_text, job_description_text)

# print("Matched Keywords:", matched_keywords)
# print("Number of Matches:", len(matched_keywords))
